In [8]:
# Lab 10 MNIST and Deep learning
import tensorflow as tf
import random
import pickle
import numpy as np
# import matplotlib.pyplot as plt

tf.reset_default_graph()
tf.set_random_seed(777)  # reproducibility

mnist = pickle.load(open('mushroom.p', 'rb'))
# Check out https://www.tensorflow.org/get_started/mnist/beginners for
# more information about the mnist dataset
_, w, h, c = mnist['train_image'].shape

def batch_index(l, batch_size):
    a = l // batch_size
    b = np.random.permutation(l)
    c = np.split(b, [(i+1) * batch_size for i in range(a)])
    if len(c[-1]) == 0:
        c = c[:-1]
    return c

# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# input place holders
X = tf.placeholder(tf.float32, [None, w * h * c])
Y = tf.placeholder(tf.float32, [None, 2])

# weights & bias for nn layers
# http://stackoverflow.com/questions/33640581/how-to-do-xavier-initialization-on-tensorflow
W1 = tf.get_variable("W1", shape=[w * h * c, 512],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

#W2 = tf.get_variable("W2", shape=[512, 512],
#                     initializer=tf.contrib.layers.xavier_initializer())
#b2 = tf.Variable(tf.random_normal([512]))
# L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# W3 = tf.get_variable("W3", shape=[512, 512],
#                      initializer=tf.contrib.layers.xavier_initializer())
# b3 = tf.Variable(tf.random_normal([512]))
# L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# W4 = tf.get_variable("W4", shape=[512, 512],
#                      initializer=tf.contrib.layers.xavier_initializer())
# b4 = tf.Variable(tf.random_normal([512]))
# L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("W5", shape=[512, 2],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L1, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

def onehot(y):
    return np.array([[1,0] if i == 0 else [0,1] for i in y])

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(mnist['train_image']) / batch_size)
    batch_ind = batch_index(len(mnist['train_image']), batch_size)
    
    for i in range(total_batch):
        batch_xs = mnist['train_image'][batch_ind[i]].reshape(-1, w*h*c)
        batch_ys = mnist['train_label'][batch_ind[i]]
        batch_ys = onehot(batch_ys)
        feed_dict = {X: batch_xs, Y: batch_ys}
        cc, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += cc / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: mnist['test_image'].reshape(-1, w*h*c), Y: onehot(mnist['test_label'])}))

# Get one and predict
r = random.randint(0, len(mnist['test_label']) - 1)
print("Label: ", mnist['test_label'][r:r + 1])
print("Prediction: ", sess.run(
    tf.argmax(hypothesis, 1), feed_dict={X: mnist['test_image'][r:r + 1].reshape(-1, w*h*c)}))

# plt.imshow(mnist.test.images[r:r + 1].
#           reshape(28, 28), cmap='Greys', interpolation='nearest')
# plt.show()

'''
Epoch: 0001 cost = 0.266061549
Epoch: 0002 cost = 0.080796588
Epoch: 0003 cost = 0.049075800
Epoch: 0004 cost = 0.034772298
Epoch: 0005 cost = 0.024780529
Epoch: 0006 cost = 0.017072763
Epoch: 0007 cost = 0.014031383
Epoch: 0008 cost = 0.013763446
Epoch: 0009 cost = 0.009164047
Epoch: 0010 cost = 0.008291388
Epoch: 0011 cost = 0.007319742
Epoch: 0012 cost = 0.006434021
Epoch: 0013 cost = 0.005684378
Epoch: 0014 cost = 0.004781207
Epoch: 0015 cost = 0.004342310
Learning Finished!
Accuracy: 0.9742
'''


Epoch: 0001 cost = 91.811946845
Epoch: 0002 cost = 1.178607333
Epoch: 0003 cost = 1.164935189
Epoch: 0004 cost = 1.150374156
Epoch: 0005 cost = 1.135684457
Epoch: 0006 cost = 1.121054885
Epoch: 0007 cost = 1.106849658
Epoch: 0008 cost = 1.092613754
Epoch: 0009 cost = 1.078749990
Epoch: 0010 cost = 1.064995146
Epoch: 0011 cost = 1.051564112
Epoch: 0012 cost = 1.038314959
Epoch: 0013 cost = 1.025417131
Epoch: 0014 cost = 1.012769824
Epoch: 0015 cost = 1.000177926
Learning Finished!
Accuracy: 0.5
Label:  [ 0.]
Prediction:  [1]


'\nEpoch: 0001 cost = 0.266061549\nEpoch: 0002 cost = 0.080796588\nEpoch: 0003 cost = 0.049075800\nEpoch: 0004 cost = 0.034772298\nEpoch: 0005 cost = 0.024780529\nEpoch: 0006 cost = 0.017072763\nEpoch: 0007 cost = 0.014031383\nEpoch: 0008 cost = 0.013763446\nEpoch: 0009 cost = 0.009164047\nEpoch: 0010 cost = 0.008291388\nEpoch: 0011 cost = 0.007319742\nEpoch: 0012 cost = 0.006434021\nEpoch: 0013 cost = 0.005684378\nEpoch: 0014 cost = 0.004781207\nEpoch: 0015 cost = 0.004342310\nLearning Finished!\nAccuracy: 0.9742\n'